In [7]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import re  
import warnings
from pathlib import Path
from tqdm import tqdm
warnings.filterwarnings("ignore")
%matplotlib inline
plt.rcParams['figure.figsize'] = [16,8]
plt.rcParams['font.size'] =14
plt.rcParams['font.weight']= 'bold'
sns.set()

In [2]:
import torch 
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader , Dataset , random_split
import torchvision.datasets as datasets 
import torchvision.transforms as transforms

In [11]:
%run dataset.ipynb import getData

train_loader , val_loader , test_loader = getData(224 ,224)



In [27]:
from torchvision.models import resnet18
%run model.ipynb import TinyVGG
from torch.optim import lr_scheduler
device = 'cuda' if torch.cuda.is_available() else 'cpu' 
#model = TinyVGG(input_shape= 3 , hidden_units=10 , output_shape=4).to(device)

model = resnet18(pretrained = False)
in_features = model.fc.in_features
model.fc = nn.Linear(in_features , 4)
model.to(device);

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(params = model.parameters() , lr = 1e-4)
lr_scheduler = lr_scheduler.StepLR(optimizer ,step_size = 4, gamma = 0.66)

In [29]:
%run Training-Testing.ipynb import *

trainingLosses , trainingAcc = [] , []
valLosses , valAcc = [] , []

for epoch in tqdm(range(20)):
    
    train_loss , train_acc = trainingStep(model , train_loader , criterion , optimizer , device)
    val_loss , val_acc = testingStep(model , val_loader , criterion , device)
    lr_scheduler.step()
    
    trainingLosses.append(train_loss)
    trainingAcc.append(train_acc)
    valLosses.append(val_loss)
    valAcc.append(val_acc)
    
    print(f'trainingLoss = {train_loss} - trainingAcc = {train_acc} | valLoss = {val_loss} - valAcc = {val_acc} ')

Unfrotunately, my laptop can't handle the computational needed even for a small model like resnet18 , so this is a base structure of a model , so i can not predicted what could happen to the model , but i will give you some tips in case of model did not work well with you .<br>
Note i tried 4 Epochs just to get sense of how maybe could be efficient,and the result was<br>
    Train_acc  |  val_acc<br>
1-  65      |     69.5<br>
2-  75.8    |     77.3<br>
3-  80      |     79.2<br>
4-  84      |     82.5<br>
So, it looks to be good , but also it can not be reliable for just 4 Epochs
<pre>
1 - change optimizer learning_rate
2 - change optimzer 
3 - add DataAugmentation for data if needed 
4 - try googleNet or any other arch (googleNet Recommended)
5 - try to train it longer = More epochs
6 - if the model overfitting ,Add Reguleraization and  try model more simpler OR
make Resnet More simpler if you can (a little bit challenging)

finally,when the model works well then Merge the train/validation dataset and train the model again on the whole dataset